# PREPARE S3DIS to training data

## libraries

In [1]:
#IMPORT PACKAGES
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import random as rd
import pandas as pd
import cv2
import laspy
import copy
from matplotlib import pyplot as plt
import math
import pathlib

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
from geomapi.utils import imageutils as iu
import geomapi.tools as tl
from geomapi.tools import progresstools as pt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [13]:
dir_in="D:/Data/Stanford3dDataset_v1.2_Aligned_Version/Stanford3dDataset_v1.2_Aligned_Version"
dir_out="D:/Data/Stanford3dDataset_v1.2_Aligned_Version/Prepared"

train_areas = ['Area_1', 'Area_2', 'Area_3', 'Area_4', 'Area_6']
# test_areas = ['Area_5']
# test_area=5
# split='train' #'test'
classes=['ceiling','floor','wall','column','beam','window','door','table','chair','bookcase','sofa','board','clutter','stairs']
movePoints=True
includeHeight=True

# Parse S3DIS data

create dataframes

In [ ]:
# get areas
area_list = sorted(os.listdir(dir_in))
area_list = [item for item in area_list if 'Area_' in item]
print(area_list)

for area in area_list[4:5]: 
    print(area)
    i=area[-1]
    df_total=pd.DataFrame()
    
    #get rooms  
    room_list = sorted([d for d in os.listdir(os.path.join(dir_in,area)) if os.path.isdir(os.path.join(os.path.join(dir_in,area), d))])
    
    for r,room in enumerate(room_list[0:5]): #! temp
        #get points
        dir=os.path.join(dir_in,area,room,'Annotations')                    
        files=[f for f in ut.get_list_of_files(dir)  if f.endswith('.txt')]
        for f in files:
            #convert name to index
            name=ut.get_filename(f,'_')
            index=classes.index(name)
            
            #read points
            df=pd.read_csv(f,sep=' ',names=['x','y','z','r','g','b']) 
            
            #optionally displace points
            if movePoints:
                columns=r%10
                rows=math.floor(r/10)
                transform=gmu.get_cartesian_transform(translation=np.array([50*rows,50*columns,0]))
                df=gmu.transform_dataframe(df,transform=transform)
            
            #add index as point field 
            df['label']=pd.DataFrame(np.full((df.shape[0],1),index))

            #combine dataframes
            df_total=pd.concat([df_total,df], ignore_index=False)
    
    # 1. Create a new header
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.add_extra_dim(laspy.ExtraBytesParams(name="R", type=np.int32))
    header.add_extra_dim(laspy.ExtraBytesParams(name="G", type=np.int32))
    header.add_extra_dim(laspy.ExtraBytesParams(name="B", type=np.int32))
    header.add_extra_dim(laspy.ExtraBytesParams(name="label", type=np.int32))
    header.add_extra_dim(laspy.ExtraBytesParams(name="height", type=np.float32)) if includeHeight else None  
    # header.offsets = np.min(df_total, axis=0)
    # header.scales = np.array([0.1, 0.1, 0.1])

    # 2. Create a Las
    las = laspy.LasData(header)
    las.x = df_total.iloc[:,0].to_numpy()
    las.y = df_total.iloc[:,1].to_numpy()
    las.z = df_total.iloc[:,2].to_numpy()
    las.red = df_total.iloc[:,3].to_numpy()
    las.green = df_total.iloc[:,4].to_numpy()
    las.blue = df_total.iloc[:,5].to_numpy()
    las.label =df_total.iloc[:,6].to_numpy()

    #(optional height)
    if includeHeight:   
        las.height=las.z-np.min(las.z)
    
    #write las
    las.write(os.path.join(dir_out,f'Area_{i}_labeled.las'))        
            
    # print(df_total.head)
    print(df_total.shape)

['Area_1', 'Area_2', 'Area_3', 'Area_4', 'Area_5', 'Area_6']
Area_5


export as las file

In [55]:
# # 1. Create a new header
# header = laspy.LasHeader(point_format=3, version="1.2")
# header.add_extra_dim(laspy.ExtraBytesParams(name="R", type=np.int32))
# header.add_extra_dim(laspy.ExtraBytesParams(name="G", type=np.int32))
# header.add_extra_dim(laspy.ExtraBytesParams(name="B", type=np.int32))
# header.add_extra_dim(laspy.ExtraBytesParams(name="label", type=np.int32))

# # header.offsets = np.min(df_total, axis=0)
# # header.scales = np.array([0.1, 0.1, 0.1])

# # 2. Create a Las
# las = laspy.LasData(header)
# las.x = df_total.iloc[:,0].to_numpy()
# las.y = df_total.iloc[:,1].to_numpy()
# las.z = df_total.iloc[:,2].to_numpy()
# las.red = df_total.iloc[:,3].to_numpy()
# las.green = df_total.iloc[:,4].to_numpy()
# las.blue = df_total.iloc[:,5].to_numpy()
# las.label =df_total.iloc[:,6].to_numpy()

# #(optional height)
# if includeHeight:   
#     las.header.add_extra_dim(laspy.ExtraBytesParams(name="height", type=np.int32))
#     las.height=las.z-np.min(las.z)

# #write las
# las.write(os.path.join(dir_out,f'Area_{5}_labeled.las'))

optional, export dataframes with labels


In [39]:
#export DataFrame to text file
with open(os.path.join(dir_out,f'Area_{5}_labeled.txt'), 'a') as f:
    df_string = df_total.to_string(header=False, index=False)
    f.write(df_string)

optionally, convert to pcd

In [19]:
pcd=gmu.dataframe_to_pcd(df_total,xyz=[0,1,2],rgb=[3,4,5])
print(pcd)

PointCloud with 5836948 points.


In [20]:
o3d.visualization.draw_geometries([pcd])

optionally, color by class

In [34]:
#get classes
classArray=df_total.iloc[:,6].to_numpy() 
colors=gmu.array_to_colors(classArray)
pcd.colors=o3d.utility.Vector3dVector(colors)

In [35]:
o3d.visualization.draw_geometries([pcd])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: De aangevraagde overdrachtsbewerking wordt niet ondersteund. 


# Features

classes=['ceiling','floor','wall','column','beam','window','door','table','chair','bookcase','sofa','board','clutter','stairs']
normals
planarity 0.2 -> floors, ceilings, walls
verticallity 0.05
surface variation 0.10
sum eigenvalues 0.1 -> goed voor deurlijsten
omnivariance 0.05 -> goed voor clutter, deuren
eigenentropy 0.02 -> good voor clutter
eigenvalue 0.1 -> medium clutter
roughness
'column','beam'
beams -> density in Z
sphericity -> half a meter?

not:
same as eigenvalues
surface variation 0.05 -> respons well for clutter (leggs of chairs)

# functions